# Xarray and parallel computing with dask

xarray integrates with dask to support parallel computations and streaming computation on datasets that don’t fit into memory.

### Outline
- reading and writing data
- automatic parallelization
- dask schedulers and deployments

### Tutorial Duriation
10 minutes

### Going Further

- Xarray's Documentation on using dask: http://xarray.pydata.org/en/latest/dask.html
- Dask's Documentation: http://dask.pydata.org/en/latest/

## Dask

Dask is a flexible parallel computing library for analytic computing.

Dask is composed of two components:

- Dynamic task scheduling optimized for computation. This is similar to Airflow, Luigi, Celery, or Make, but optimized for interactive computational workloads.
- “Big Data” collections like parallel arrays, dataframes, and lists that extend common interfaces like NumPy, Pandas, or Python iterators to larger-than-memory or distributed environments. These parallel collections run on top of the dynamic task schedulers.

![](images/collections-schedulers.png)


Dask works by constructing "task graphs"

![](images/transpose.svg)


## Reading and Writing data

The usual way to create a dataset filled with dask arrays is to load the data from a netCDF file or files. You can do this by supplying a chunks argument to open_dataset() or using the open_mfdataset() function.

In [1]:
%matplotlib inline

import xarray as xr
import numpy as np

np.warnings.filterwarnings('ignore')


/opt/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
# use dask's distributed scheduler for now
# (see the last section of this notebook for more details)
from distributed import Client, progress
client = Client()
client

Client Scheduler: tcp://127.0.0.1:35865 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 8 Cores: 8 Memory: 31.63 GB


In [3]:
ds = xr.open_dataset('./data/rasm.nc', chunks={'time': 10})  # this actually loads data using xr.open_dataset
ds

<xarray.Dataset>
Dimensions:  (time: 36, x: 275, y: 205)
Coordinates:
  * time     (time) datetime64[ns] 1980-09-16T12:00:00 1980-10-17 ...
    xc       (y, x) float64 dask.array<shape=(205, 275), chunksize=(205, 275)>
    yc       (y, x) float64 dask.array<shape=(205, 275), chunksize=(205, 275)>
Dimensions without coordinates: x, y
Data variables:
    Tair     (time, y, x) float64 dask.array<shape=(36, 205, 275), chunksize=(10, 205, 275)>
Attributes:
    title:                     /workspace/jhamman/processed/R1002RBRxaaa01a/l...
    institution:               U.W.
    source:                    RACM R1002RBRxaaa01a
    output_frequency:          daily
    output_mode:               averaged
    convention:                CF-1.4
    references:                Based on the initial model of Liang et al., 19...
    comment:                   Output from the Variable Infiltration Capacity...
    nco_openmp_thread_number:  1
    NCO:                       "4.6.0"
    history:              

In [4]:
# xarray is now using dask arrays under the hood
ds['Tair'].data

dask.array<open_dataset-c78a78e4379a70dda2c502894f4eee29Tair, shape=(36, 205, 275), dtype=float64, chunksize=(10, 205, 275)>

In [5]:
## Xarray operations using dask are now lazy
t_range = (ds['Tair'].resample(time='AS').max('time') -
           ds['Tair'].resample(time='AS').min('time')).mean('time')

In [7]:
t_range.data.visualize()

RuntimeError: Drawing dask graphs requires the `graphviz` python library and the `graphviz` system library to be installed.

In [8]:
results = t_range.persist()
progress(results)

VBox()

tornado.application - ERROR - Future <tornado.concurrent.Future object at 0x7fc719e53748> exception was never retrieved: Traceback (most recent call last):
  File "/opt/conda/lib/python3.6/site-packages/tornado/gen.py", line 1063, in run
    yielded = self.gen.throw(*exc_info)
  File "/opt/conda/lib/python3.6/site-packages/distributed/core.py", line 351, in handle_comm
    yield comm.write(result, serializers=serializers)
  File "/opt/conda/lib/python3.6/site-packages/tornado/gen.py", line 1055, in run
    value = future.result()
  File "/opt/conda/lib/python3.6/site-packages/tornado/concurrent.py", line 238, in result
    raise_exc_info(self._exc_info)
  File "<string>", line 4, in raise_exc_info
  File "/opt/conda/lib/python3.6/site-packages/tornado/gen.py", line 1069, in run
    yielded = self.gen.send(value)
  File "/opt/conda/lib/python3.6/site-packages/distributed/comm/tcp.py", line 229, in write
    stream.write(b)
  File "/opt/conda/lib/python3.6/site-packages/tornado/iostream.

In [9]:
results

<xarray.DataArray 'Tair' (y: 205, x: 275)>
dask.array<shape=(205, 275), dtype=float64, chunksize=(205, 275)>
Coordinates:
    xc       (y, x) float64 dask.array<shape=(205, 275), chunksize=(205, 275)>
    yc       (y, x) float64 dask.array<shape=(205, 275), chunksize=(205, 275)>
Dimensions without coordinates: y, x

tornado.application - ERROR - Future <tornado.concurrent.Future object at 0x7fc719e5c2e8> exception was never retrieved: Traceback (most recent call last):
  File "/opt/conda/lib/python3.6/site-packages/tornado/gen.py", line 1063, in run
    yielded = self.gen.throw(*exc_info)
  File "/opt/conda/lib/python3.6/site-packages/distributed/core.py", line 351, in handle_comm
    yield comm.write(result, serializers=serializers)
  File "/opt/conda/lib/python3.6/site-packages/tornado/gen.py", line 1055, in run
    value = future.result()
  File "/opt/conda/lib/python3.6/site-packages/tornado/concurrent.py", line 238, in result
    raise_exc_info(self._exc_info)
  File "<string>", line 4, in raise_exc_info
  File "/opt/conda/lib/python3.6/site-packages/tornado/gen.py", line 1069, in run
    yielded = self.gen.send(value)
  File "/opt/conda/lib/python3.6/site-packages/distributed/comm/tcp.py", line 229, in write
    stream.write(b)
  File "/opt/conda/lib/python3.6/site-packages/tornado/iostream.

## Automatic parallelization

Almost all of xarray’s built-in operations work on dask arrays. If you want to use a function that isn’t wrapped by xarray, one option is to extract dask arrays from xarray objects (.data) and use dask directly.

Another option is to use xarray’s apply_ufunc(), which can automate embarrassingly parallel “map” type operations where a functions written for processing NumPy arrays should be repeatedly applied to xarray objects containing dask arrays. It works similarly to `dask.array.map_blocks()` and `dask.array.atop()`, but without requiring an intermediate layer of abstraction.

In [10]:
import bottleneck

def covariance_gufunc(x, y):
    return ((x - x.mean(axis=-1, keepdims=True))
            * (y - y.mean(axis=-1, keepdims=True))).mean(axis=-1)

def pearson_correlation_gufunc(x, y):
    return covariance_gufunc(x, y) / (x.std(axis=-1) * y.std(axis=-1))

def spearman_correlation_gufunc(x, y):
    x_ranks = bottleneck.rankdata(x, axis=-1)
    y_ranks = bottleneck.rankdata(y, axis=-1)
    return pearson_correlation_gufunc(x_ranks, y_ranks)

def spearman_correlation(x, y, dim):
    return xr.apply_ufunc(
        spearman_correlation_gufunc, x, y,
        input_core_dims=[[dim], [dim]],
        dask='parallelized',
        output_dtypes=[float])

distributed.utils - ERROR - Existing exports of data: object cannot be re-sized
Traceback (most recent call last):
  File "/opt/conda/lib/python3.6/site-packages/distributed/utils.py", line 646, in log_errors
    yield
  File "/opt/conda/lib/python3.6/site-packages/distributed/scheduler.py", line 2886, in feed
    yield comm.write(response)
  File "/opt/conda/lib/python3.6/site-packages/tornado/gen.py", line 1055, in run
    value = future.result()
  File "/opt/conda/lib/python3.6/site-packages/tornado/concurrent.py", line 238, in result
    raise_exc_info(self._exc_info)
  File "<string>", line 4, in raise_exc_info
  File "/opt/conda/lib/python3.6/site-packages/tornado/gen.py", line 1069, in run
    yielded = self.gen.send(value)
  File "/opt/conda/lib/python3.6/site-packages/distributed/comm/tcp.py", line 229, in write
    stream.write(b)
  File "/opt/conda/lib/python3.6/site-packages/tornado/iostream.py", line 406, in write
    self._handle_write()
  File "/opt/conda/lib/python3.6/s

In [11]:
ds['Tair'].roll(time=1)

<xarray.DataArray 'Tair' (time: 36, y: 205, x: 275)>
dask.array<shape=(36, 205, 275), dtype=float64, chunksize=(10, 205, 275)>
Coordinates:
  * time     (time) datetime64[ns] 1983-08-17 1980-09-16T12:00:00 1980-10-17 ...
    xc       (y, x) float64 dask.array<shape=(205, 275), chunksize=(205, 275)>
    yc       (y, x) float64 dask.array<shape=(205, 275), chunksize=(205, 275)>
Dimensions without coordinates: y, x
Attributes:
    units:           C
    long_name:       Surface air temperature
    type_preferred:  double
    time_rep:        instantaneous

tornado.application - ERROR - Exception in callback functools.partial(<function wrap.<locals>.null_wrapper at 0x7fc719db3ae8>, <tornado.concurrent.Future object at 0x7fc719c89d30>)
Traceback (most recent call last):
  File "/opt/conda/lib/python3.6/site-packages/tornado/ioloop.py", line 605, in _run_callback
    ret = callback()
  File "/opt/conda/lib/python3.6/site-packages/tornado/stack_context.py", line 277, in null_wrapper
    return fn(*args, **kwargs)
  File "/opt/conda/lib/python3.6/site-packages/tornado/ioloop.py", line 626, in _discard_future_result
    future.result()
  File "/opt/conda/lib/python3.6/site-packages/tornado/concurrent.py", line 238, in result
    raise_exc_info(self._exc_info)
  File "<string>", line 4, in raise_exc_info
  File "/opt/conda/lib/python3.6/site-packages/tornado/gen.py", line 1063, in run
    yielded = self.gen.throw(*exc_info)
  File "/opt/conda/lib/python3.6/site-packages/distributed/client.py", line 894, in _update_scheduler_info
    self._sched

In [12]:
ds = ds.chunk({'x': 25, 'y': 25, 'time': -1})  # rechunk this dataset

# create some toy data using our dataset
x = ds['Tair']
y = ds['Tair'].roll(time=1)  # lag the air temp data by one month
y['time'] = x['time']  # reset the index 


# calculate the spearman correlation
corr = spearman_correlation(x, y, 'time')
corr = corr.where(corr < 1)  # make sure nans are masked properly
corr

tornado.application - ERROR - Future <tornado.concurrent.Future object at 0x7fc719dfbc18> exception was never retrieved: Traceback (most recent call last):
  File "/opt/conda/lib/python3.6/site-packages/tornado/gen.py", line 1063, in run
    yielded = self.gen.throw(*exc_info)
  File "/opt/conda/lib/python3.6/site-packages/distributed/core.py", line 351, in handle_comm
    yield comm.write(result, serializers=serializers)
  File "/opt/conda/lib/python3.6/site-packages/tornado/gen.py", line 1055, in run
    value = future.result()
  File "/opt/conda/lib/python3.6/site-packages/tornado/concurrent.py", line 238, in result
    raise_exc_info(self._exc_info)
  File "<string>", line 4, in raise_exc_info
  File "/opt/conda/lib/python3.6/site-packages/tornado/gen.py", line 1069, in run
    yielded = self.gen.send(value)
  File "/opt/conda/lib/python3.6/site-packages/distributed/comm/tcp.py", line 229, in write
    stream.write(b)
  File "/opt/conda/lib/python3.6/site-packages/tornado/iostream.

KeyboardInterrupt: 

distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
tornado.application - ERROR - Future <tornado.concurrent.Future object at 0x7fc719ad9160> exception was never retrieved: Traceback (most recent call last):
  File "/opt/conda/lib/python3.6/site-packages/tornado/gen.py", line 1063, in run
    yielded = self.gen.throw(*exc_info)
  File "/opt/conda/lib/python3.6/site-packages/distributed/core.py", line 351, in handle_comm
    yield comm.write(result, serializers=serializers)
  File "/opt/conda/lib/python3.6/site-packages/tornado/gen.py", line 1055, in run
    value = future.result()
  File "/opt/conda/lib/python3.6/site-packages/tornado/concurrent.py", line 238,

tornado.application - ERROR - Future <tornado.concurrent.Future object at 0x7fc76c19c320> exception was never retrieved: Traceback (most recent call last):
  File "/opt/conda/lib/python3.6/site-packages/tornado/gen.py", line 1063, in run
    yielded = self.gen.throw(*exc_info)
  File "/opt/conda/lib/python3.6/site-packages/distributed/core.py", line 351, in handle_comm
    yield comm.write(result, serializers=serializers)
  File "/opt/conda/lib/python3.6/site-packages/tornado/gen.py", line 1055, in run
    value = future.result()
  File "/opt/conda/lib/python3.6/site-packages/tornado/concurrent.py", line 238, in result
    raise_exc_info(self._exc_info)
  File "<string>", line 4, in raise_exc_info
  File "/opt/conda/lib/python3.6/site-packages/tornado/gen.py", line 1069, in run
    yielded = self.gen.send(value)
  File "/opt/conda/lib/python3.6/site-packages/distributed/comm/tcp.py", line 229, in write
    stream.write(b)
  File "/opt/conda/lib/python3.6/site-packages/tornado/iostream.

tornado.application - ERROR - Future <tornado.concurrent.Future object at 0x7fc719c2b9b0> exception was never retrieved: Traceback (most recent call last):
  File "/opt/conda/lib/python3.6/site-packages/tornado/gen.py", line 1063, in run
    yielded = self.gen.throw(*exc_info)
  File "/opt/conda/lib/python3.6/site-packages/distributed/core.py", line 351, in handle_comm
    yield comm.write(result, serializers=serializers)
  File "/opt/conda/lib/python3.6/site-packages/tornado/gen.py", line 1055, in run
    value = future.result()
  File "/opt/conda/lib/python3.6/site-packages/tornado/concurrent.py", line 238, in result
    raise_exc_info(self._exc_info)
  File "<string>", line 4, in raise_exc_info
  File "/opt/conda/lib/python3.6/site-packages/tornado/gen.py", line 1069, in run
    yielded = self.gen.send(value)
  File "/opt/conda/lib/python3.6/site-packages/distributed/comm/tcp.py", line 229, in write
    stream.write(b)
  File "/opt/conda/lib/python3.6/site-packages/tornado/iostream.

...

tornado.application - ERROR - Future <tornado.concurrent.Future object at 0x7fc76c4a9278> exception was never retrieved: Traceback (most recent call last):
  File "/opt/conda/lib/python3.6/site-packages/tornado/gen.py", line 1063, in run
    yielded = self.gen.throw(*exc_info)
  File "/opt/conda/lib/python3.6/site-packages/distributed/core.py", line 351, in handle_comm
    yield comm.write(result, serializers=serializers)
  File "/opt/conda/lib/python3.6/site-packages/tornado/gen.py", line 1055, in run
    value = future.result()
  File "/opt/conda/lib/python3.6/site-packages/tornado/concurrent.py", line 238, in result
    raise_exc_info(self._exc_info)
  File "<string>", line 4, in raise_exc_info
  File "/opt/conda/lib/python3.6/site-packages/tornado/gen.py", line 1069, in run
    yielded = self.gen.send(value)
  File "/opt/conda/lib/python3.6/site-packages/distributed/comm/tcp.py", line 229, in write
    stream.write(b)
  File "/opt/conda/lib/python3.6/site-packages/tornado/iostream.

tornado.application - ERROR - Future <tornado.concurrent.Future object at 0x7fc76c35b7f0> exception was never retrieved: Traceback (most recent call last):
  File "/opt/conda/lib/python3.6/site-packages/tornado/gen.py", line 1063, in run
    yielded = self.gen.throw(*exc_info)
  File "/opt/conda/lib/python3.6/site-packages/distributed/core.py", line 351, in handle_comm
    yield comm.write(result, serializers=serializers)
  File "/opt/conda/lib/python3.6/site-packages/tornado/gen.py", line 1055, in run
    value = future.result()
  File "/opt/conda/lib/python3.6/site-packages/tornado/concurrent.py", line 238, in result
    raise_exc_info(self._exc_info)
  File "<string>", line 4, in raise_exc_info
  File "/opt/conda/lib/python3.6/site-packages/tornado/gen.py", line 1069, in run
    yielded = self.gen.send(value)
  File "/opt/conda/lib/python3.6/site-packages/distributed/comm/tcp.py", line 229, in write
    stream.write(b)
  File "/opt/conda/lib/python3.6/site-packages/tornado/iostream.

tornado.application - ERROR - Future <tornado.concurrent.Future object at 0x7fc76c33cef0> exception was never retrieved: Traceback (most recent call last):
  File "/opt/conda/lib/python3.6/site-packages/tornado/gen.py", line 1063, in run
    yielded = self.gen.throw(*exc_info)
  File "/opt/conda/lib/python3.6/site-packages/distributed/core.py", line 351, in handle_comm
    yield comm.write(result, serializers=serializers)
  File "/opt/conda/lib/python3.6/site-packages/tornado/gen.py", line 1055, in run
    value = future.result()
  File "/opt/conda/lib/python3.6/site-packages/tornado/concurrent.py", line 238, in result
    raise_exc_info(self._exc_info)
  File "<string>", line 4, in raise_exc_info
  File "/opt/conda/lib/python3.6/site-packages/tornado/gen.py", line 1069, in run
    yielded = self.gen.send(value)
  File "/opt/conda/lib/python3.6/site-packages/distributed/comm/tcp.py", line 229, in write
    stream.write(b)
  File "/opt/conda/lib/python3.6/site-packages/tornado/iostream.

tornado.application - ERROR - Future <tornado.concurrent.Future object at 0x7fc76c29a390> exception was never retrieved: Traceback (most recent call last):
  File "/opt/conda/lib/python3.6/site-packages/tornado/gen.py", line 1063, in run
    yielded = self.gen.throw(*exc_info)
  File "/opt/conda/lib/python3.6/site-packages/distributed/core.py", line 351, in handle_comm
    yield comm.write(result, serializers=serializers)
  File "/opt/conda/lib/python3.6/site-packages/tornado/gen.py", line 1055, in run
    value = future.result()
  File "/opt/conda/lib/python3.6/site-packages/tornado/concurrent.py", line 238, in result
    raise_exc_info(self._exc_info)
  File "<string>", line 4, in raise_exc_info
  File "/opt/conda/lib/python3.6/site-packages/tornado/gen.py", line 1069, in run
    yielded = self.gen.send(value)
  File "/opt/conda/lib/python3.6/site-packages/distributed/comm/tcp.py", line 229, in write
    stream.write(b)
  File "/opt/conda/lib/python3.6/site-packages/tornado/iostream.

tornado.application - ERROR - Future <tornado.concurrent.Future object at 0x7fc76c25cb38> exception was never retrieved: Traceback (most recent call last):
  File "/opt/conda/lib/python3.6/site-packages/tornado/gen.py", line 1063, in run
    yielded = self.gen.throw(*exc_info)
  File "/opt/conda/lib/python3.6/site-packages/distributed/core.py", line 351, in handle_comm
    yield comm.write(result, serializers=serializers)
  File "/opt/conda/lib/python3.6/site-packages/tornado/gen.py", line 1055, in run
    value = future.result()
  File "/opt/conda/lib/python3.6/site-packages/tornado/concurrent.py", line 238, in result
    raise_exc_info(self._exc_info)
  File "<string>", line 4, in raise_exc_info
  File "/opt/conda/lib/python3.6/site-packages/tornado/gen.py", line 1069, in run
    yielded = self.gen.send(value)
  File "/opt/conda/lib/python3.6/site-packages/distributed/comm/tcp.py", line 229, in write
    stream.write(b)
  File "/opt/conda/lib/python3.6/site-packages/tornado/iostream.

In [ ]:
corr.plot()

## dask schedulers and deployments

Dask provides 4 different mechanisms for executing dask task graphs.

- `dask.threaded.get`: a scheduler backed by a thread pool
- `dask.multiprocessing.get`: a scheduler backed by a process pool
- `dask.get`: a synchronous scheduler, good for debugging
- `distributed.Client.get`: a distributed scheduler for executing graphs

In [ ]:
import dask.multiprocessing
dask.config.set(scheduler='processes')  # overwrite default with multiprocessing scheduler

# or

from distributed import Client
client = Client()

## Lots more! 

- Setting up dask: http://dask.pydata.org/en/latest/setup.html